In [7]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np
import concurrent.futures
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
service = Service(executable_path=r'C:\Users\DELL\Documents\web_scraping\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service)


driver.get("https://greenbook.nafdac.gov.ng/")
driver.implicitly_wait(10)

try:
    product_category = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//select[@id='product_categtory']")))
    product_category.click()

    product_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//option[contains(text(), 'Vaccines and Biologics')]")))
    product_option.click()

    time.sleep(3)
    
    number_output = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//select[@class='custom-select custom-select-sm form-control form-control-sm']")))
    number_output.click()

    choose_100 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//option[contains(text(), '100')]")))
    choose_100.click()

except Exception as e:
    print(e)

time.sleep(3)


# Locate the table element (you might need to inspect the webpage to find the correct selector)
table = driver.find_element(By.XPATH, "//table[@class='table table-bordered data-table  dataTable no-footer']")

# Extract table rows
heads = table.find_elements(By.TAG_NAME, "tr")

# Extract data from each row and store it in a list of lists
data = []
for row in heads:
    cells = row.find_elements(By.TAG_NAME, "th")
    row_data = [cell.text for cell in cells]
    data.append(row_data)


table2 = driver.find_element(By.XPATH, "//table[@class='table table-bordered data-table  dataTable no-footer dtr-inline collapsed']")

# Extract table rows
rows = table2.find_elements(By.TAG_NAME, "tr")

# Extract data from each row and store it in a list of lists
data2 = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    data2.append(row_data)   

# wait = WebDriverWait(driver, 10)

# Wait for element to be clickable
# element = wait.until(EC.element_to_be_clickable((By.ID, "button")))

element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//ul//li//a[contains(text(),'2')]"))
)

# Scroll to the element:
actions = ActionChains(driver)
actions.move_to_element(element).perform()

# Click using JavaScript:
driver.execute_script("arguments[0].click();", element)


# Wait for the next page to load
WebDriverWait(driver, 10,).until(EC.presence_of_element_located((By.XPATH, "//table[@class='table table-bordered data-table  dataTable no-footer dtr-inline collapsed']")))

# Extract data from the second page (200 rows)
table3 = driver.find_element(By.XPATH, "//table[@class='table table-bordered data-table  dataTable no-footer dtr-inline collapsed']")


row_count = len(table3.find_elements(By.TAG_NAME, "tr"))
i = 0
while i < row_count:
    time.sleep(3)
    try:
        row = table3.find_elements(By.TAG_NAME, "tr")[i]
        cells1 = row.find_elements(By.TAG_NAME, "td")  
        row_data1 = [cell.text for cell in cells1]  
        data2.append(row_data1)
        i += 1  
    except IndexError:
        row_count = len(table3.find_elements(By.TAG_NAME, "tr"))  # Update row count

# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(data2,columns=data)

# Print the DataFrame (or save it to a file)
print(df)

# Close the web driver
driver.quit()






                                          Product Name Active Ingredients  \
0                                                 None               None   
1                                        Actemra 162mg        Tocilizumab   
2                                        Actemra 200mg        Tocilizumab   
3                                        Actemra 400mg        Tocilizumab   
4                                         Actemra 80mg        Tocilizumab   
..                                                 ...                ...   
106  Wosulin N Suspension for Injection 100 IU/mL V...   Isophane insulin   
107      Wosulin R Insulin Human Injection Cartridge**    Soluble insulin   
108                Wosulin R Insulin Human Injection**    Soluble insulin   
109                                 ZARZIO 30MU/0.5ML#         Filgrastim   
110                                 ZARZIO 48MU/0.5ML#         Filgrastim   

           Product Category        NRN                      Form  \
0      

In [18]:
df.to_csv('nafdac_data.csv')